<a href="https://colab.research.google.com/github/OmerElshrief/Fine-Grained-Action-Detection-Recognition/blob/master/Multi_Stream_bi_Directional_LSTM_for_Video_action_detection_%26_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dependencies

In [0]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from torch.autograd import Variable
import numpy as np;
from torch.utils.data import Dataset, DataLoader
import math;
import torch.optim as optim
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('grayscale')
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, models, transforms
import time
#print("PyTorch Version: ",torch.__version__)
#print("Torchvision Version: ",torchvision.__version__)
import torchvision.models as models
from torchsummary import summary
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import PIL
from PIL import Image
import scipy
import scipy.io
import torch.utils.data as utils
from itertools import cycle,chain,repeat
import itertools
import tensorflow as tf

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
torch.cuda.is_available()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

## Data Loader

In [0]:


# def __init__( start_id , threshold_id, videos_path,labels_path):
#   self.subject_id = start_id
#   self.threshold = threshold_id
#   self.videos_path = videos_path
#   self.session_id = 1
#   self.labels_path = labels_path



def write_video(video,out_path, frame_width, frame_height,subject_id, session_id, as_numpy = False):
  if as_numpy:
    path = out_path +str(subject_id) + "_"+str(session_id)+".npy"
    np.save(path,np.array(video),allow_pickle=False)
  else:

    path = out_path +str(subject_id) + "_"+str(session_id)+".avi"
    out = cv2.VideoWriter(path,1, 20.0, frameSize = (frame_width,frame_height),)
    print("Video lenght: ", len(video))
    for frame in video:
      # Write the frame into the file 'output.avi'
      out.write(frame)
    out.release()
    del out
  print('Video of lenght ',str(len(video)),' is saved at ',path)


  
def get_data(videos_path,label_path, sample_labels = True,fps = 15, denoising = False, blur = True, resize = True,
                normalize = False, frame_width = 256, frame_height=256, background_sub = False, object_trajectory = False, 
                data_from_numpy = False):
  """
  To load the Dataset, Dataset  can be loaded from Videos or from Stored Numpy arrays 
  Parameters:
  videos_path: Path of the Dataset
  Label_path: path of the Labels 
  sample_labels: If true, labels will be sampled, label per 6 frames 
  background_sub: If true, frames loaded from videos will be background subtracted (for the Background subtraction models)
  object_trajectory: If true, frames loaded from video will be used to extract the Object trajectory for each 6 frames (for Temporal Models)
  data_from_numpy: determing if the Dataset is loaded from videos or from Stored Numpy arrays
  """

  data  , length= load_video(videos_path,fps = fps, denoising = denoising,blur = blur,resize = resize,normalize = normalize,frame_width = frame_width,
                             frame_height = frame_height,background_sub = background_sub,object_trajectory = object_trajectory,
                             data_from_numpy = data_from_numpy )
  # print('loaded ',path)
  # labels = np.load(labels_path,allow_pickle= True)
  labels = get_video_label(label_path,length,half = True,data_from_numpy=data_from_numpy)
  if not object_trajectory:
    ## We take a sample frame each 6 frames
    print(data.shape)
    data = data[[i for i in range(0,len(data),6)]]
  print(data.shape)
  data = [prepare(i) for i in data]
  data = torch.stack(data)
  print(len(labels))
  if sample_labels:
    # Since each chunk of video is 6 Frames, we sample the Labels at 6 labels per sample
    labels = labels[[i for i in range(0,len(labels),6)]]
  ## Transform the Data
  labels = torch.tensor(labels)
  print(len(labels))
  count =len(data)
  # print(labels.shape)
  # print(data.shape)
  ## Sometime labels lenght might not be equal frames lenght,
  while (len(labels) > len(data)):
    labels = labels[:-1]
    print('removed from labels')
  while (len(labels) < len(data)):
    data = data[:-1]
    print('removed from Data')
  return data , labels, count


def load_video(video_path, fps = 15, denoising = False, blur = True, resize = True, normalize = False, frame_width = 256, frame_height=256, 
               background_sub = False,object_trajectory = False, data_from_numpy = False):
      """
      background_sub: If true, frames loaded from videos will be background subtracted (for the Background subtraction models)
      object_trajectory: If true, frames loaded from video will be used to extract the Object trajectory for each 6 frames
      data_from_numpy: determing if the Dataset is loaded from videos or from Stored Numpy arrays
      """
      video  = []
      print('Loading ',video_path)
      if data_from_numpy:
         video = np.load(path)
         length = len(video)

      else:
          cap = cv2.VideoCapture(video_path)
          length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
          
          # taking the Background from 2nd second 
          cap.set(cv2.CAP_PROP_POS_MSEC,2*1000)
          reval, background = cap.read()
          # Preprocess the Background
          background = cv2.GaussianBlur(background, (3,3), 0) 
          # Get back to the 0 second
          cap.set(cv2.CAP_PROP_POS_MSEC,0)
          while True:
#                 video.append(img)
              if object_trajectory: # If it's desired to Load video frames and get the object trajectory for each 6 consecutive frames
                 chunk = []
                 for i in range(0,6):
                    reval, img = cap.read() # This is neglected because we load at 15 FPS , and video is stored at 30 FPS
                    reval, img = cap.read()

                    if not reval:
                        # End of frames
                        print('END')
                        video = np.array(video)
                        return video , length
                    if background_sub:      # This is needed for the 4th Model, as we subtract the BG and then get OT
                        # Subtract Background
                        img = subtract_background(img,background)
                    img = cv2.resize(img, (512, 512))
                    chunk.append(img) # Here we store video chunk of 6 Frames and then pass it to get_stacked_pixel_trajectory
                 chunk = np.array(chunk)
                 video.append(get_stacked_pixel_trajectory(chunk,2))

              else:

                  reval, img = cap.read() # Video is stored at 30 FPS, we want to load it at 15 FPS so we ignore a frame each iteration
                  reval, img = cap.read() # Faster than cap.set()

                  if not reval:
                      # End of frames
                      break
                  if background_sub:
                    # Subtract Background
                    img = subtract_background(img,background)

                  if resize:
                    img = cv2.resize(img, (frame_width, frame_height)) 
                  if blur:
                    img = cv2.GaussianBlur(img, (21,21), 0) 
                  if normalize:
                    img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                  
                  video.append(img)

          video = np.array(video)
          # try:
          #   video = video[[i for i in range(0,len(video),2)]]
          # except IndexError:
            # print('index error')
            # pass
          cap.release()
          del cap
          print(length)
      return video , length

    
def get_video_label(label_path,video_lenght, half = True, data_from_numpy = False):
  """
  Load the labels from matlab files or form Numpy files
  """
  if data_from_numpy :
    labels = np.load(path)

  else:

    label_data = scipy.io.loadmat(label_path)
    # Initially each frame is label 0 
    label = [0 for i in range(video_lenght)] 
    label = np.array(label)

    for category_number in range(5):
        # Each video chunk in the same class defined as in labels
        for video_chunk in label_data['tlabs'][category_number][0]:
          label[video_chunk[0]:video_chunk[1]] = category_number+1

    label = np.array(label)
    if half:
        label = label[[i for i in range(1,video_lenght,2)]]

  return (label)













## Preprocessor

In [0]:
transform = transforms.Compose(
    
    [
#         transforms.ToPILImage(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
     ])


def prepare(img):
  
  # img = cv2.fastNlMeansDenoisingColored(img,None,15,10,7,21)
  # img = cv2.GaussianBlur(img, (3,3), 0) 
  img = transform(img)
  
  # img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  
  return img
  
def get_optical_flow(first_frame,second_frame, rgb ):
        '''
        To extract the Obtical flow between 2 frames 
        '''
        # Converts frame to grayscale because we only need the luminance channel for detecting edges - less computationally expensive
        original = first_frame
        if rgb:
         
          first_frame = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
          second_frame = cv2.cvtColor(second_frame, cv2.COLOR_BGR2GRAY)
        else:
           original = cv2.cvtColor(original, cv2.COLOR_GRAY2BGR)
        # denoising and bluring
        # first_gray = cv2.fastNlMeansDenoising(first_gray,None,templateWindowSize= 7,h = 10,searchWindowSize  = 21)
        
        # second_gray = cv2.fastNlMeansDenoising(second_gray,None,templateWindowSize= 7,h = 10,searchWindowSize  = 21)
        # Creates an image filled with zero intensities with the same dimensions as the frame
        mask = np.zeros_like(original)
        # Sets image saturation to maximum
        mask[..., 1] = 255
        flow = cv2.calcOpticalFlowFarneback(first_frame, second_frame, None, 0.5, 3, 15, 3, 5, 1.2, 0)
        # Computes the magnitude and angle of the 2D vectors
        magnitude, angle = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        # Sets image hue according to the optical flow direction
        mask[..., 0] = angle * 180 / np.pi / 2
        # Sets image value according to the optical flow magnitude (normalized)
        mask[..., 2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX)
        # Converts HSV to RGB (BGR) color representation
        rgb = cv2.cvtColor(mask, cv2.COLOR_HSV2BGR)
        return cv2.resize(rgb, (224, 224))


  
def get_stacked_pixel_trajectory( video1, i,rgb = True):
    '''
    Function to get stacked optical flows between a Middle from video[i] and it's 6 neighbouring frames
    to form the Object trajectory 
    video: chunk of frames [7 frames]
    i : index of the middel frame
    '''
#     print("Extracting Obejct trajectory")
    of1 = get_optical_flow(first_frame = video1[i], second_frame=video1[i-3],rgb = rgb )
    of2 = get_optical_flow(video1[i], video1[i-2],rgb)
    of3 = get_optical_flow(video1[i],video1[i-1],rgb)

    of4 = get_optical_flow(video1[i],video1[i+1],rgb)
    of5 = get_optical_flow(video1[i],video1[i+2],rgb)
    of6 = get_optical_flow(video1[i],video1[i+3],rgb)

    of = np.concatenate((of1, of2, of3, of4, of5, of6), axis = 1)
#     plt.figure()
#    plt.imshow(of)
    return of


  
  
  
def subtract_background(frame,background, List = False):
    
    if List:
      chunk = []
      for img in frame:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # In each iteration, calculate absolute difference between current frame and reference frame
        difference = cv2.absdiff(gray, background)
        # Apply thresholding to eliminate noise
        thresh = cv2.threshold(difference, 15, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.dilate(thresh, None, iterations=2)
        thresh = cv2.cvtColor(thresh,cv2.COLOR_GRAY2BGR)
        chunk.append(thresh)
      return chunk

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # In each iteration, calculate absolute difference between current frame and reference frame
    difference = cv2.absdiff(gray, background)
    # Apply thresholding to eliminate noise
    thresh = cv2.threshold(difference, 15, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=2)
    thresh = cv2.cvtColor(thresh,cv2.COLOR_GRAY2BGR)
    return(thresh)









def label_to_one_hot(labels):
    batch_size = len(labels)
    nb_digits = 6
    # Dummy input that HAS to be 2D for the scatter (you can use view(-1,1) if needed)
    y = torch.Tensor(labels.float()).view(-1,1).long()
    # One hot encoding buffer that you create out of the loop and just keep reusing
    y_onehot = torch.FloatTensor(batch_size, nb_digits)

    # In your for loop
    y_onehot.zero_()
    y_onehot.scatter_(1, y, 1)

    return y_onehot
      

## Evaluation Functions 

In [0]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from itertools import cycle


def evaluate_average_percision(Y_test, y_score, n_classes):
    # For each class
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(n_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                            y_score[:, i])
        average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
        y_score.ravel())
    average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                         average="micro")
    return precision,recall,average_precision
    
def print_Average_precision(precision, recall, average_precision, n_classes, plot_classes = False):
    
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))

    plt.figure()
    plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall["micro"], precision["micro"], alpha=0.2, color='b'    )

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title(
        'Average precision score, micro-averaged over all classes: AP={0:0.2f}'
        .format(average_precision["micro"]))
    
    if plot_classes:
      colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal','teal'])
      plt.figure(figsize=(7, 8))
      f_scores = np.linspace(0.2, 0.8, num=4)
      lines = []
      labels = []
      for f_score in f_scores:
          x = np.linspace(0.01, 1)
          y = f_score * x / (2 * x - f_score)
          l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
          plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

      lines.append(l)
      labels.append('iso-f1 curves')
      l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
      lines.append(l)
      labels.append('micro-average Precision-recall (area = {0:0.2f})'
                    ''.format(average_precision["micro"]))

      for i, color in zip(range(n_classes), colors):
          l, = plt.plot(recall[i], precision[i], color=color, lw=2)
          lines.append(l)
          labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                        ''.format(i, average_precision[i]))

      fig = plt.gcf()
      fig.subplots_adjust(bottom=0.25)
      plt.xlim([0.0, 1.0])
      plt.ylim([0.0, 1.05])
      plt.xlabel('Recall')
      plt.ylabel('Precision')
      plt.title('Extension of Precision-Recall curve to multi-class')
      plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))
      
      
def evaluation(on_hot_predictions, ground_truth):
  """"
  input: one hot encoded predictions, ground truth labels 
  output: list of 5 precisions for each class 
  """""
  
  predictions = np.argmax(on_hot_predictions, axis = 1)
  pred = []
  for idx, _ in enumerate(predictions): 
    pred.append(np.dot(np.where(predictions == idx), 6))
  
  labels = []
  for category in ground_truth:
    labels.append(list(map(lambda x: list(range(x[0],x[1])), category[0])))
  
  true = []
  for idx, _ in enumerate(ground_truth):
    true.append(list(map(lambda x: np.sum(np.isin(pred[idx][0], x)), labels[idx])))
  
  true_positives = []
  for idx, _ in enumerate(ground_truth):
    true_positives.append(np.sum(np.array(true[idx]) != 0))
    
  false_positives = []
  for idx, _ in enumerate(ground_truth):
    false_positives.append(len(pred[idx][0]) - true_positives[idx])
  
  precison = np.array(true_positives) / (np.array(true_positives) + np.array(false_positives))
  return precison 


## Avidbeam Evaluation 

In [0]:
# This is an implementation of the mAP metric calculation over classes 1-5 (ignoring class 0)

def evaluate(result, gt):
  assert(len(result) == len(gt)), 'input arrays must have the same size'
  
  # saving true positives (tp) counts and false positives (fp) counts
  tp = {'1' : 0, '2' : 0, '3' : 0, '4' : 0, '5' : 0}
  fp = {'1' : 0, '2' : 0, '3' : 0, '4' : 0, '5' : 0}
  
  for index in range(0, len(gt)):
    # ignoring class 0
    if gt[index] != 0 and result[index] != 0:
      if gt[index] == result[index]:
        tp[str(gt[index])] += 1
      else:
        fp[str(result[index])] += 1

  # calculating precision per class as tp/(tp + fp)
  AP = {k : tp[k]/(tp[k] + fp[k]) for k in tp if k in fp}
  print ('precision per class =', AP)
  mAP = float(sum(AP.values())) / len(AP)
  print ('mAP =', mAP)
  return mAP
      


## Helper Fucntions 

In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False
    else:
      for param in model.parameters():
            param.requires_grad = True
      
    
      
def create_optimizer(model , r = 0.0001):
  params_to_update = model.parameters()
  print("Params to learn:")
  
  params_to_update = []
  for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
        
  return  optim.SGD(params_to_update, lr=r,weight_decay=1e-6)

      
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
      
class Transform(nn.Module):
  def __init__(self):
    super(Transform, self).__init__()
  
  def forward(self,x):
    return transforms(x)
        

## Models

In [0]:
import torchvision.models as models
vgg16 = models.vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 553433881/553433881 [00:09<00:00, 58894800.29it/s]


In [0]:
def prepare_models(spatial,temporal,spatial2,temporal2,path_to_weights_dir):
  weights = torch.load(path_to_weights_dir+"/model1.pi")
  spatial.load_state_dict(weights)
  weights = torch.load(path_to_weights_dir+"/model_temporal.pt")
  temporal.load_state_dict(weights)
#   weights = torch.load(path_to_weights_dir+"/sub_back_patial_model12.pi")
#   spatial2.load_state_dict(weights)
#   weights= torch.load(path_to_weights_dir+"/model_temporal_background_subtracted1.pt")
#   temporal2.load_state_dict(weights)
  del weights
  torch.cuda.empty_cache()

  return spatial,temporal


class backbone_feature_extractor (nn.Module):
  def __init__(self,features_extraction_output,softmax_output = False, classification = False,  Dropout = 0.5):
    
        super().__init__()
        
        self.features_extractor = vgg16.features
        set_parameter_requires_grad(self.features_extractor,True)
#         set_parameter_requires_grad(self.features_extractor[24:31],False)
        self.flatten = Flatten()
        self.classifier = nn.Sequential(nn.Linear(in_features = features_extraction_output ,out_features=4096, bias=True ),
                                        nn.ReLU(),
                                        nn.Dropout(Dropout),
                                        nn.Linear(in_features=4096, out_features=1024, bias=True),
                                        nn.ReLU(),
                                        nn.Dropout(Dropout),
                                        nn.Linear(in_features=1024, out_features=6, bias=True)
                                       )
        set_parameter_requires_grad(self.classifier,True)                                
        self.classification = classification
  
  def forward(self, x):
    torch.cuda.empty_cache()

    x = self.features_extractor(x)
    torch.cuda.empty_cache()

    x = self.flatten(x)
    if self.classification:
      
      x= self.classifier(x)
      
    return x



class MSN(nn.Module):
  
  """
  Takes a chunked video of shape (n_chunks, frames, frame_height, frame_width, channels)
  
  Outputs a tensor of shape (n_chunks, 200) containing chunk feature vector
  """
  
  def __init__(self,spatial,temporal,spatial2,temporal2,features_extraction_output):
    super().__init__()
#     weights = torch.load(path_to_weights_dir+"/model_temporal.pt")
#     keys = ['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias', 'classifier.6.weight','classifier.6.bias']
#     for key in keys:
#     weights.pop(key)  ## We only need Feature extractors weights
    
    self.cnn_temp_coarse = temporal2
#     self.cnn_temp_fine   = temporal2
    self.cnn_spat_coarse = spatial
#     self.cnn_spat_fine   = spatial

    self.fin_vec = nn.Sequential(nn.Linear(in_features =features_extraction_output , out_features = 2048),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(in_features = 2048 , out_features= 200)
                                )
   
                                 
    self.flatten = Flatten()
    
  def forward(self,x):
    
    x1 = (self.cnn_spat_coarse(x[1]))
    x2 = (self.cnn_temp_coarse(x[2]))
#     x3 = (self.cnn_spat_fine(x[3]))
#     x4 = (self.cnn_temp_fine(x[4]))
    o = torch.cat((x1, x2), dim=1)
    
    o = self.flatten(o)
    torch.cuda.empty_cache()
    o = self.fin_vec(o)
    torch.cuda.empty_cache()
    
    return o




class bi_lstm(nn.Module):

  def __init__(self,msn,batch_size,hidden_size=60,num_layers = 1):
    super().__init__()
    self.msn_feature_extractor = msn
    self.lstm = nn.LSTM(200, hidden_size= hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True )

#     self.lstm.cuda()
    
    # h_0, c_0 of shape (num_layers * num_directions, batch, hidden_size)
    
    self.hidden_vect_1 = hidden_vect_1 = (Variable(torch.zeros(2*num_layers, batch_size, hidden_size).cuda()),
                                          Variable(torch.zeros(2*num_layers, batch_size, hidden_size).cuda()))
    self.forward_classifier = nn.Linear(in_features = 60, out_features = 6)
    self.backward_classifier = nn.Linear(in_features = 60, out_features = 6)

  def forward(self,x):
    torch.cuda.empty_cache()

    x = self.msn_feature_extractor(x)   ## Shape = (n_chunks, 200)
    torch.cuda.empty_cache()

    x = x.view(-1, 1, 200)
    
    # output of shape (seq_len, batch, num_directions * hidden_size)
    output, hidden = self.lstm( x, self.hidden_vect_1    )
    torch.cuda.empty_cache()

    x_forward, x_backward = torch.split(output,60,dim=2)
  
  
    x_forward= (self.forward_classifier(x_forward))
    torch.cuda.empty_cache()

    x_backward = (self.backward_classifier(x_backward))
    torch.cuda.empty_cache()

    x = (x_forward+x_backward) /2
    

    return (x)

  
  
  


## Manual cells LSTM

In [0]:
class Model(nn.Module):
  
  """
  Takes a chunked video of shape (n_chunks, frames, channels, frame_height, frame_width)
                         
  Outputs 2 tensors of shape (n_chunks, 2) labels containing the action prediction of chunks
  using backward and forward lstm
  """
  
  def __init__(self,msn):
    super().__init__()
    self.msn_feature_extractor = msn
    self.lstm_cell = nn.LSTM(input_size = 200, hidden_size = 60,num_layers=1, batch_first=False,bidirectional= True)
    self.classifier = nn.Linear(in_features = 60, out_features = 6)
    self.fin = nn.Softmax(dim = 1)
    
    
  def forward(self,x, get_logits = True):
    x = self.msn_feature_extractor(x)   ## Shape = (n_chunks, 200)
    x = x.view(-1, 1, 200)
    
    hx = torch.randn(1, 60)                ## Handle shape
    cx = torch.randn(1, 60)                ## Handle shape
    forward_outputs = []
    print(x.shape)
    # for chunk in range(x.shape[0]):
    hx, cx = self.lstm_cell(x, (hx, cx))
    forward_outputs.append(hx)
      
    hx = torch.randn(1, 60)                ## Handle shape
    cx = torch.randn(1, 60)                ## Handle shape   
    backward_outputs = []
    # for chunk in reversed(range(x.shape[0])):
    hx, cx = self.lstm_cell(x, (hx, cx))
    backward_outputs.append(hx)
      
    forward_outputs = torch.FloatTensor(forward_outputs)      ## Handle Shape
    backward_outputs = torch.FloatTensor(backward_outpus)     ## Handle Shape
    
    pre_logits_forw = self.classifier(forward_outputs)
    pre_logits_back = self.classifier(backward_outputs)
    
    if get_logits:
      logits_forw = self.fin(pre_logits_forw)
      logits_back = self.fin(pre_logits_back)
      return torch.mean(torch.stack([logits_forw, logits_back])) 
    
    return pre_logits_forw, pre_logits_back   

## Training & Testing 

In [0]:

def model_fit(data_path, label_path, from_numpy, model_type, no_training_subjects, no_validation_subjects,
          batch_size,classification_model,save_path  ,
          optimizer, no_epochs=5 ,fps=15 ,criterion =  nn.CrossEntropyLoss()):
    '''
    This function performs the following
    1- load the Dataset (call get_data) according to the Type of model and Type of Data set (NUMPY of Video)
    2- Construct the Data Loader 
    3- Perform training step (call train function)
    4- Perform validation step
    Parameters:
    from_numpy (boolean) if true, the data is loaded from numpy arrays
    '''
    val_acc = -999
    history = []
    validation_subjects = no_training_subjects+1 + no_validation_subjects
    
    if from_numpy:
      data_extension = '.npy'
      label_extension = '_label.npy'
      
    else:
      data_extension = '_crop.mp4'
      label_extension = '_label.mat'
      if model_type ==3:
        background_sub = True
   
         
           
        
    for i in range(no_epochs):
        print('Training Epoch {} ...'.format(i+1))
        running_loss,train_correct,valid_correct,val_loss = 0,0,0,0
        session_id, subject_id = 1,1
        train_count,valid_count = 0,0
        while subject_id < no_training_subjects+1:

            # Loading the videos and labels one at a time from the stored Numpy files
            path =  data_path + "/"+ str(subject_id) + "_" + str(session_id)+ data_extension
            if  os.path.exists(path): # check whether this file exists in the directory
              labels_path =  label_path + str(subject_id) +"_" + str(session_id)+label_extension
              
              data , labels, count = get_data(path,labels_path,True , background_sub = background_sub,data_from_numpy = from_numpy)
              train_count += count
              print(data.shape)
              print(labels.shape)
              my_dataset = utils.TensorDataset(data,labels) # create your datset
              train_dataloader = utils.DataLoader(my_dataset,batch_size=batch_size, shuffle=False) # 
              del data, labels, my_dataset # To free some CUDA Memory
              # Training Step
              loss , correct = training_batch(classification_model,train_dataloader,optimizer,criterion)
              running_loss +=loss
              train_correct += correct
            else:
                print('No such path:' ,path)
            # Next Video    
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
           
        ## Validation step
        print( " Validating Epoch ", i+1)
        subject_id,session_id = no_training_subjects+1 , 1
        
        while subject_id < validation_subjects:

            # Loading the videos and labels one at a time from the stored Numpy files or from Stored Videos
            path =  data_path + "/"+ str(subject_id) + "_" + str(session_id)+"_crop.mp4"
            if  os.path.exists(path): # check whether this file exists in the directory
              labels_path =  label_path + str(subject_id) +"_" + str(session_id)+"_label.mat"
              
              data , labels, count = get_data(path,labels_path,True , background_sub = background_sub,data_from_numpy = from_numpy)
              valid_count += count
              my_dataset = utils.TensorDataset(data,labels) # create your datset
              train_dataloader = utils.DataLoader(my_dataset,batch_size=batch_size, shuffle=False) # 
              del data, labels, my_dataset # To free some CUDA Memory
              # Validation Step
              loss , correct = validation_step(classification_model,train_dataloader,criterion)
              val_loss += loss
              valid_correct += correct
            # Next Video
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
        
        test_accu =  int(train_correct) / train_count
        valid_accu = int(valid_correct) / valid_count

        print('Train correct: ',int(train_correct), ' out of: ',train_count )
        print('Tets correct: ', int(valid_correct), ' out of: ',valid_count)
        print ('Epoch [{}/{}], -T-Loss : {:.6f} , Train_accuracy: {:.4f} ,  Val_loss: {:.6f}  -Val_acc: {:.4f}'.format(
            i+1, no_epochs,running_loss/train_count,test_accu ,val_loss/valid_count, valid_accu))

        if val_acc < valid_accu:
          torch.save(classification_model.state_dict(), save_path)
          print('val_acc has improved from {:.4f} to {:.4f}, model is saved at {}'.format(val_acc , valid_accu,save_path))
          val_acc = valid_accu
        else:
          print('val_acc has not Improved from {:.4f}'.format(val_acc))    
        history.append([i,running_loss/train_count,valid_accu,val_loss/valid_count])

        #accu.append(100 * int(test_correct) / len(X_test))

    return history



def training_batch(classification_model,train_dataloader,optimizer,criterion):
      running_loss,train_correct = 0,0
      probability = nn.Softmax()
      for batch,y_true in train_dataloader:
          y_true= y_true.type(torch.LongTensor)
          y_true = y_true.cuda()
          batch = (batch.cuda())
          optimizer.zero_grad()
          y_pred = classification_model(batch)
          loss = criterion(y_pred, y_true)
          loss.backward()
          optimizer.step()
          running_loss += loss.item()
          y_pred = probability(y_pred)
          train_correct +=  (y_true == y_pred.max(1)[1]).sum()
          del loss, batch
          y_pred.cpu()
          y_true.cpu()
          torch.cuda.empty_cache()
      return running_loss,train_correct

def validation_step(classification_model,train_dataloader,criterion):
  val_loss,valid_correct = 0,0
  for batch,y_true in train_dataloader:
      y_true= y_true.type(torch.LongTensor)
      y_true = y_true.cuda()
      batch = (batch.cuda())
      with torch.no_grad():
        y_pred = classification_model(batch)
        y_pred = y_pred.view(-1,6)
        valid_correct +=  (y_true == y_pred.max(1)[1]).sum()
        loss = criterion(y_pred, y_true)
        val_loss += loss.item()
        del batch
        y_pred.cpu()
        y_true.cpu()
        torch.cuda.empty_cache()
  return val_loss, valid_correct


def test(classification_model, model_type, data_path, label_path, data_from_numpy,subject_id, test_subjects, batch_size,
         background_sub = False, criterion =  nn.CrossEntropyLoss()):
    '''
    Parameters:
    subject_id: initial subject_id, the starting point
    test_subjects: Number of test subjects
    '''
    probability = nn.Softmax()
    test_count,valid_correct,valid_count = 0,0,0
    threshold = subject_id + test_subjects
    session_id = 1
    while subject_id < threshold:

        # Loading the videos and labels one at a time from the stored Numpy files
        # path = data_path + str(subject_id)+"_"+str(session_id)+"_fps_"+str(fps)+'.npy'
       # Loading the videos and labels one at a time from the stored Numpy files or from Stored Videos
        path =  data_path + "/"+ str(subject_id) + "_" + str(session_id)+"_crop.mp4"
        if  os.path.exists(path): # check whether this file exists in the directory
          labels_path =  label_path + str(subject_id) +"_" + str(session_id)+"_label.mat"

          data , labels, count = get_data(path,labels_path,True , background_sub = background_sub,data_from_numpy = from_numpy)
          test_count += count
          my_dataset = utils.TensorDataset(data,labels) # create your datset
          train_dataloader = utils.DataLoader(my_dataset,batch_size=batch_size, shuffle=False) # 
          del data, labels, my_dataset # To free some CUDA Memory

          score = []
          predictions =[]
          for batch,y_true in train_dataloader:
              y_true= y_true.type(torch.LongTensor)
              y_true = y_true.cuda()
              batch = (batch.cuda())
              with torch.no_grad():
                y_pred = classification_model(batch)
                y_pred = y_pred.view(-1,6)

                valid_correct +=  (y_true == y_pred.max(1)[1]).sum()
                loss = criterion(y_pred, y_true)
                val_loss += loss.item()
                y_pred = probability(y_pred)
                y_pred = y_pred.cpu().numpy()
                
                y_pred = list(chain.from_iterable(repeat(n, 6) for n in y_pred))
                predictions.append(np.array(y_pred))
          
          predictions = np.array(predictions)
          predictions = predictions.reshape(predictions.shape[0]*predictions.shape[1],6)
          print(predictions.shape)
          print(len(original_labels))
          
          while len(original_labels) > len(predictions):
            original_labels = original_labels[:-1]
          while len(original_labels) < len(predictions):
            predictions = predictions[:-1]
          print(len(predictions))
          print(len(original_labels))
          score.append(evaluate_average_percision(label_to_one_hot(original_labels).numpy(),predictions,6))
          del batch
         
          torch.cuda.empty_cache()
        session_id += 1
        if session_id > 3:
          subject_id += 1
          session_id = 1

    avg_precision =0
    for i in range(len(score)):
           avg_precision += score[i][2]['micro']
    avg_precision = avg_precision / len(score)
                             
    valid_accu = int(valid_correct) / valid_count
    print('Tess Results: ', int(valid_correct), ' out of: ',valid_count)
    print('Accuracy: ',valid_accu)
    print('Loss', str(val_loss / valid_count))
    print('average precision: ',avg_precision)
    return score
  

  




## Full Model Functions

In [0]:

def get_data_MS(videos_path,label_path, scaling_factor = 0.5,sample_labels = True ):
  
  
  # data  , length= load_video_MS(videos_path,scaling_factor )
  print('Loading ',videos_path)
  data = np.load(videos_path,allow_pickle=True)
  length = len(data) *12
  # print('loaded ',path)
  # labels = np.load(labels_path,allow_pickle= True)
  labels = get_video_label_MS(label_path,length,half = True)
  if sample_labels:
    # Since each chunk of video is 6 Frames, we sample the Labels at 6 labels per sample
    labels = labels[[i for i in range(0,len(labels),6)]]
    while (len(labels) > len(data)):
      labels = labels[:-1]
      print('removed from labels')
    while (len(labels) < len(data)):
      labels.append(labels[len(labels)-1])
      print('Added to Data')

  labels = torch.tensor(labels)
  print(len(labels))
  count =len(data)
  return data , labels, count

  ## Transform the Data
  labels = torch.tensor(labels)
  print(len(labels))
  count =len(data)
  # print(labels.shape)
  # print(data.shape)
  ## Sometime labels lenght might not be equal frames lenght,

    
def get_video_label_MS(label_path,video_lenght, half = True, data_from_numpy = False):
  

  
  label_data = scipy.io.loadmat(label_path)
  # Initially each frame is label 0 
  label = [0 for i in range(video_lenght)] 
  label = np.array(label)

  for category_number in range(5):
      # Each video chunk in the same class defined as in labels
      for video_chunk in label_data['tlabs'][category_number][0]:
        label[video_chunk[0]:video_chunk[1]] = category_number+1

  label = np.array(label)
  if half:
      label = label[[i for i in range(1,video_lenght,2)]]

  return (label)




def load_video_MS(video_path, resize_scale = 0.5, half = True):
     
          video  = []
          print('Loading ',video_path)
     
          cap = cv2.VideoCapture(video_path)
          length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
          frame_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
          frame_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
          print('Frame width: '+str(frame_width) + "   frame_height: "+str(frame_height))
          width ,height= int(frame_width * resize_scale), int(frame_height*resize_scale)
          print('resizing_width: '+str(width) + '    resizing_heighth: '+str(height))
          # taking the Background from 2nd second 
          cap.set(cv2.CAP_PROP_POS_MSEC,2*1000)
          reval, background = cap.read()
          # Preprocess the Background
          background = cv2.resize(background, (width, height))
          background = cv2.GaussianBlur(background, (21,21), 0) 
          background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
          # Get back to the 0 second
          cap.set(cv2.CAP_PROP_POS_MSEC,0 * 1000)
          while True:
#                 video.append(img)
              chunk = []
              for i in range(0,6):
                  if half:
                    reval, img = cap.read() # This is neglected because we load at 15 FPS , and video is stored at 30 FPS
                  reval, img = cap.read()

                  if not reval:
                      # End of frames
                      if i != 0:
                        while i < 6 :
                          chunk.append(chunk[len(chunk)-1])
                          i +=1
                          print(len(chunk))
                        print('END')
                        chunk = np.array(chunk)
                        video.append(process_chunk(chunk,background))
                      video = np.array(video)
                      cap.release()
                      del cap
                      print(length)
                      return video , length

                  img = cv2.resize(img, (width, height))
                  img = cv2.GaussianBlur(img,(3,3),0)
                  chunk.append(img) # Here we store video chunk of 6 Frames and then pass it to get_stacked_pixel_trajectory
              chunk = np.array(chunk)
              video.append(process_chunk(chunk,background))
          video = np.array(video)
          cap.release()
          del cap
          print(length)
          return video , length


def process_chunk(chunk,background):
  img1 = cv2.resize( chunk[2],(256,256))
  img2 = get_stacked_pixel_trajectory(chunk,2)
  img3 = cv2.resize(subtract_background(chunk[2],background),(256,256))
  img4 = get_stacked_pixel_trajectory(subtract_background(chunk,background,List = True),2,rgb = True)
  chunk = [img1,img2,img3,img4]
  return chunk

def get_data_loader(video,labels,batch_size, with_labels  = True):
  x1 = [torch.tensor(video[i][0]) for i in range(len(video))]
  x1 = [Image.fromarray(i.numpy()) for i in x1]
  x1 = [prepare(i) for i in x1]
  x1 = torch.stack(x1, dim = 0)

  x2 = [torch.tensor(video[i][1]) for i in range(len(video))]
  x2 = [Image.fromarray(i.numpy()) for i in x2]
  x2 = [prepare(i) for i in x2]
  x2 = torch.stack(x2, dim = 0)

  x3 = [torch.tensor(video[i][2]) for i in range(len(video))]
  x3 = [Image.fromarray(i.numpy()) for i in x3]
  x3 = [prepare(i) for i in x3]
  x3 = torch.stack(x3, dim = 0)

  x4 = [torch.tensor(video[i][3]) for i in range(len(video))]
  x4 = [Image.fromarray(i.numpy()) for i in x4]
  x4 = [prepare(i) for i in x4]
  x4 = torch.stack(x4, dim = 0)
  
  
  
  if with_labels :
    my_dataset = utils.TensorDataset(x1,x2,x3,x4,labels) # create your datset
  else:
     my_dataset = utils.TensorDataset(x1,x2,x3,x4)   
  dataloader = utils.DataLoader(my_dataset,batch_size=batch_size, shuffle=False, drop_last=True)
  return dataloader


In [0]:
def model_fit_MS(data_path, label_path,  no_training_subjects, no_validation_subjects, batch_size,classification_model,save_path  ,
          optimizer, no_epochs=5 ,fps=15 ,criterion =  nn.CrossEntropyLoss()):
    '''
    This function performs the following
    1- load the Dataset (call get_data) according to the Type of model and Type of Data set (NUMPY of Video)
    2- Construct the Data Loader 
    3- Perform training step (call train function)
    4- Perform validation step
    Parameters:
    from_numpy (boolean) if true, the data is loaded from numpy arrays
    '''
    val_acc = -999
    history = []
    validation_subjects = no_training_subjects+1 + no_validation_subjects
    session_id, subject_id = 1,1
    for i in range(no_epochs):
        print('Training Epoch {} ...'.format(i+1))
        running_loss,train_correct,valid_correct,val_loss = 0,0,0,0
        
        train_count,valid_count = 0,0
        while subject_id < no_training_subjects+1:

            # Loading the videos and labels one at a time from the stored Numpy files
            path =  data_path + "/Copy of "+ str(subject_id) + "_" + str(session_id)+ '.npy'
            if  os.path.exists(path): # check whether this file exists in the directory
              labels_path =  label_path + str(subject_id) +"_" + str(session_id)+'_label.mat'
              data , labels, count = get_data_MS(path,labels_path )
              train_count += count
             
             
              train_dataloader = get_data_loader(data,labels,batch_size=batch_size)
              # del data, labels, my_dataset # To free some CUDA Memory
              # Training Step
              loss , correct = training_batch(classification_model,train_dataloader,optimizer,criterion)
              torch.cuda.empty_cache()
              running_loss +=loss
              train_correct += correct
            else:
                print('No such path:' ,path)
            # Next Video    
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
              if subject_id > 10:
                 torch.save(classification_model.state_dict(), save_path)
         
           
        ## Validation step
        print( " Validating Epoch ", i+1)
        subject_id,session_id = no_training_subjects+1 , 1
        
        while subject_id < validation_subjects:

            # Loading the videos and labels one at a time from the stored Numpy files or from Stored Videos
            path =  data_path + "/Copy of "+ str(subject_id) + "_" + str(session_id)+ '.npy'
            if  os.path.exists(path): # check whether this file exists in the directory
              labels_path =  label_path + str(subject_id) +"_" + str(session_id)+"_label.mat"
              
              data , labels, count = get_data_MS(path,labels_path)
              valid_count += count
              train_dataloader = get_data_loader(data,labels,batch_size=batch_size)
              # del data, labels, my_dataset # To free some CUDA Memory
              # Validation Step
              loss , correct = validation_step(classification_model,train_dataloader,criterion)
              torch.cuda.empty_cache()

              val_loss += loss
              valid_correct += correct
            # Next Video
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
        
        test_accu =  int(train_correct) / train_count
        valid_accu = int(valid_correct) / valid_count

        print('Train correct: ',int(train_correct), ' out of: ',train_count )
        print('Tets correct: ', int(valid_correct), ' out of: ',valid_count)
        print ('Epoch [{}/{}], -T-Loss : {:.6f} , Train_accuracy: {:.4f} ,  Val_loss: {:.6f}  -Val_acc: {:.4f}'.format(
            i+1, no_epochs,running_loss/train_count,test_accu ,val_loss/valid_count, valid_accu))

        if val_acc < valid_accu:
          torch.save(classification_model.state_dict(), save_path)
          print('val_acc has improved from {:.4f} to {:.4f}, model is saved at {}'.format(val_acc , valid_accu,save_path))
          val_acc = valid_accu
        else:
          print('val_acc has not Improved from {:.4f}'.format(val_acc))    
        history.append([i,running_loss/train_count,valid_accu,val_loss/valid_count])
        session_id, subject_id = 1,1
        #accu.append(100 * int(test_correct) / len(X_test))

    return history



def training_batch(classification_model,train_dataloader,optimizer,criterion):
      running_loss,train_correct = 0,0
      probability = nn.Softmax()
      for x1,x2,x3,x4,y_true in train_dataloader:
          y_true= y_true.type(torch.LongTensor)
          y_true = y_true.cuda()
          optimizer.zero_grad()
          
          x = {1:x1.cuda() , 2:x2.cuda() , 3:x3.cuda() , 4:x4.cuda()}
          y_pred = classification_model(x)
          del x
          
          torch.cuda.empty_cache()
          y_pred = y_pred.view(-1,6)
          loss = criterion(y_pred, y_true)
          running_loss += loss.item()
          y_pred = probability(y_pred)
          train_correct +=  (y_true == y_pred.max(1)[1]).sum()
          del y_pred
          
          torch.cuda.empty_cache()
          
          loss.backward()
          del loss
          optimizer.step()
          
      return running_loss,train_correct

def validation_step(classification_model,train_dataloader,criterion):
  val_loss,valid_correct = 0,0
  probability = nn.Softmax()
  for x1,x2,x3,x4,y_true in train_dataloader:
      y_true= y_true.type(torch.LongTensor)
      y_true = y_true.cuda()
      
      with torch.no_grad():
        
        x = {1:x1.cuda() , 2:x2.cuda() , 3:x3.cuda() , 4:x4.cuda()}
        
        
        y_pred = classification_model.forward(x)
        del x
        torch.cuda.empty_cache()

        y_pred = y_pred.view(-1,6)
        loss = criterion(y_pred, y_true)
        val_loss += loss.item()
        y_pred = probability(y_pred)
        valid_correct +=  (y_true == y_pred.max(1)[1]).sum()
        torch.cuda.empty_cache()
  return val_loss, valid_correct


## Models Training 

Difference between training base models is the parameters of the Training functions and the
model.
#####  For the spatial models: (Type 1)
* Data is loaded form videos, so data-from_numpy is False 
* We don't need Object_trajectory 
* For the background subtraction model, we need background_sub to be True
* Features_Extraction output = 32768 

##### For the Temporal models: (Type 2)
* Data is loaded from Numpy files, so data_from_numpy is True 
* Object_trajectory is True 
* For the background subtraction temporal model, we need background_sub to be True
* Features_Extraction output = 150528

##### Spatial background subtraction model: (Type 3)
* Same as Type 1

##### Temporal background Subtraction model: (Type 4)
* Same as Type 2

##### For the Full MSBL Model :
* Feature_Extraction output = 366592


In [0]:
#

# del msbl_model
# # torch.cuda.empty_cache()

torch.cuda.empty_cache()
path_to_weights = 'drive/My Drive/Neurons/MERL Dataset/'
model1 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
model2 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
model3 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
model4 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
model1, model2 = prepare_models(model1, model2, model3, model4,path_to_weights)
torch.cuda.empty_cache()
msn = MSN(spatial =model1, temporal=model2, spatial2=model3, temporal2=model4,        
          features_extraction_output=183296)
msbl_model = bi_lstm(msn,batch_size = 20,num_layers = 1)
# print(msbl_model)
torch.cuda.empty_cache()
weights = torch.load('drive/My Drive/MERL Dataset/Full Model2.pi')
torch.cuda.empty_cache()

msbl_model.load_state_dict(weights)
del weights 
torch.cuda.empty_cache()

msbl_model.to(device)
data_path = 'drive/My Drive/MERL Dataset/Data/'
labels_path = 'drive/My Drive/MERL Dataset/Labels_MERL_Shopping_Dataset/'

save_path = 'drive/My Drive/MERL Dataset/Full Model2.pi'
optimizer = create_optimizer(msbl_model,r =  0.001)  
history = model_fit_MS(data_path=data_path ,label_path=labels_path,batch_size=20,no_training_subjects=21,
                    no_validation_subjects=7,no_epochs=5,classification_model=msbl_model, 
                    save_path = save_path,optimizer= optimizer)


Params to learn:
	 msn_feature_extractor.fin_vec.0.weight
	 msn_feature_extractor.fin_vec.0.bias
	 msn_feature_extractor.fin_vec.3.weight
	 msn_feature_extractor.fin_vec.3.bias
	 lstm.weight_ih_l0
	 lstm.weight_hh_l0
	 lstm.bias_ih_l0
	 lstm.bias_hh_l0
	 lstm.weight_ih_l0_reverse
	 lstm.weight_hh_l0_reverse
	 lstm.bias_ih_l0_reverse
	 lstm.bias_hh_l0_reverse
	 forward_classifier.weight
	 forward_classifier.bias
	 backward_classifier.weight
	 backward_classifier.bias
Training Epoch 1 ...
Loading  drive/My Drive/MERL Dataset/Data//Copy of 1_1.npy
327


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loading  drive/My Drive/MERL Dataset/Data//Copy of 1_2.npy
323
Loading  drive/My Drive/MERL Dataset/Data//Copy of 1_3.npy
442
Loading  drive/My Drive/MERL Dataset/Data//Copy of 2_1.npy
317
Loading  drive/My Drive/MERL Dataset/Data//Copy of 2_2.npy
316
Loading  drive/My Drive/MERL Dataset/Data//Copy of 2_3.npy
299
Loading  drive/My Drive/MERL Dataset/Data//Copy of 3_1.npy
308
Loading  drive/My Drive/MERL Dataset/Data//Copy of 3_2.npy
312
Loading  drive/My Drive/MERL Dataset/Data//Copy of 3_3.npy
293
Loading  drive/My Drive/MERL Dataset/Data//Copy of 4_1.npy
331
Loading  drive/My Drive/MERL Dataset/Data//Copy of 4_2.npy
323
Loading  drive/My Drive/MERL Dataset/Data//Copy of 4_3.npy
315
Loading  drive/My Drive/MERL Dataset/Data//Copy of 5_1.npy
311
Loading  drive/My Drive/MERL Dataset/Data//Copy of 5_2.npy
330
Loading  drive/My Drive/MERL Dataset/Data//Copy of 5_3.npy
353
Loading  drive/My Drive/MERL Dataset/Data//Copy of 6_1.npy
317
Loading  drive/My Drive/MERL Dataset/Data//Copy of 6_2.

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:145: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Loading  drive/My Drive/MERL Dataset/Data//Copy of 22_2.npy
357
Loading  drive/My Drive/MERL Dataset/Data//Copy of 22_3.npy
351
Loading  drive/My Drive/MERL Dataset/Data//Copy of 23_1.npy
349
Loading  drive/My Drive/MERL Dataset/Data//Copy of 23_2.npy
441
Loading  drive/My Drive/MERL Dataset/Data//Copy of 23_3.npy
514
Loading  drive/My Drive/MERL Dataset/Data//Copy of 24_1.npy
353
Loading  drive/My Drive/MERL Dataset/Data//Copy of 24_2.npy
392
Loading  drive/My Drive/MERL Dataset/Data//Copy of 24_3.npy
407
Loading  drive/My Drive/MERL Dataset/Data//Copy of 25_1.npy
363
Loading  drive/My Drive/MERL Dataset/Data//Copy of 25_2.npy
372
Loading  drive/My Drive/MERL Dataset/Data//Copy of 25_3.npy
340
Loading  drive/My Drive/MERL Dataset/Data//Copy of 26_1.npy
376
Loading  drive/My Drive/MERL Dataset/Data//Copy of 26_2.npy
397
Loading  drive/My Drive/MERL Dataset/Data//Copy of 26_3.npy
340
Loading  drive/My Drive/MERL Dataset/Data//Copy of 27_1.npy
339
Loading  drive/My Drive/MERL Dataset/Dat

## Test


In [0]:
def test_step(classification_model,train_dataloader):
  val_loss,valid_correct = 0,0
  probability = nn.Softmax()
  predictions = []
  for x1,x2,x3,x4,y_true in train_dataloader:
      y_true= y_true.type(torch.LongTensor)
      y_true = y_true.cuda()
      
      with torch.no_grad():
        x = {1:x1.cuda() , 2:x2.cuda() , 3:x3.cuda() , 4:x4.cuda()}
        y_pred = classification_model.forward(x)
        del x
        torch.cuda.empty_cache()
        y_pred = y_pred.view(-1,6)
        y_pred = probability(y_pred)
        y_pred = y_pred.max(1)[1]
        
        valid_correct +=  (y_true == y_pred).sum()
        torch.cuda.empty_cache()
        y_pred = y_pred.cpu()
        y_pred = list(chain.from_iterable(repeat(n, 6) for n in y_pred))
        predictions.append((y_pred))
  return test_correct, predictions

def predict(classification_model,data_path, labels_path, subject_id, no_testSubjects, batch_size):
   test_correct = 0
   threshold = subject_id + no_testSubjects+1
   results, predictions, gt = [],[],[]
  
   while subject_id < threshold:

            # Loading the videos and labels one at a time from the stored Numpy files
            path =  data_path +  str(subject_id) + "_" + str(session_id)+ '.npy'
            if  os.path.exists(path): # check whether this file exists in the directory
              labels_path =  label_path + str(subject_id) +"_" + str(session_id)+'_label.mat'
              data , original_labels, count = get_data_MS(path,labels_path,sample_labels = False )
              labels = original_labels[[i for i in range(0,len(original_labels),6)]]
              while (len(labels) > len(data)):
                list(labels).remove(labels[len(labels)-1])
                print('removed from labels')
              while (len(labels) < len(data)):
                print('Added To labels')
                list(labels).append(labels[len(labels)-1])
              
              
              labels = torch.tensor(labels)
             
              train_dataloader = get_data_loader(data,labels,batch_size=batch_size)
              correct,predictions = test_step(classification_model,train_dataloader,optimizer,criterion)
              torch.cuda.empty_cache()
              
              test_correct += correct
              predictions = np.array(predictions)
              print(predictions.shape)
              predictions = predictions.reshape(predictions.shape[0]*predictions.shape[1])
              print(predictions.shape)
              print(len(original_labels))
              
              while len(original_labels) > len(predictions):
                original_labels = original_labels[:-1]
              while len(original_labels) < len(predictions):
                predictions = predictions[:-1]
              print(len(predictions))
              print(len(original_labels))
              gt.append((original_labels))
              results.append((predictions))
            else:
                print('No such path:' ,path)
            # Next Video    
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
    
    
   gt = list(itertools.chain.from_iterable(gt))
   gt = [i.item() for i in gt]
   results =  list(itertools.chain.from_iterable(results))
   mav = evaluate(results,gt)  
   print('average precision: ',mav)
   return mav

In [0]:
import itertools
classification_model =msbl_model 
subject_id = 32
session_id = 1
no_testSubjects = 10
batch_size = 20
data_path = 'drive/My Drive/MERL Dataset/Data/'
label_path = 'drive/My Drive/MERL Dataset/Labels_MERL_Shopping_Dataset/'
test_correct = 0
threshold = subject_id + no_testSubjects+1
results, predictions, gt = [],[],[]

while subject_id < threshold:

        # Loading the videos and labels one at a time from the stored Numpy files
        path =  data_path +  str(subject_id) + "_" + str(session_id)+ '.npy'
        if  os.path.exists(path): # check whether this file exists in the directory
          labels_path =  label_path + str(subject_id) +"_" + str(session_id)+'_label.mat'
          data , original_labels, count = get_data_MS(path,labels_path, sample_labels = False )
          
          labels = original_labels[[i for i in range(0,len(original_labels),6)]]
          while len(labels) > len(data):
                labels = labels[:-1]
          while len(labels) < len(data):
                data = data[:-1]
          
          
          labels = torch.tensor(labels)
          
          train_dataloader = get_data_loader(data,labels,batch_size=batch_size)
          correct,predictions = test_step(classification_model,train_dataloader)
          torch.cuda.empty_cache()
          
          test_correct += correct
          predictions = np.array(predictions)
          predictions = list(predictions.reshape(predictions.shape[0]*predictions.shape[1]))
          print('Len of original labels: ',len(original_labels))
          print('Len of  predictions: ',len(predictions))
          while len(original_labels) > len(predictions):
            original_labels = original_labels[:-1]
            print('Removed from abels')
          while len(original_labels) < len(predictions):
            list(original_labels).append(original_labels[len(original_labels)-1])
            print('Added to Labels')
          print(len(predictions))
          print(len(original_labels))
          gt.append((original_labels))
          predictions = np.array(predictions)
          results.append((predictions))
        else:
            print('No such path:' ,path)
        # Next Video    
        session_id += 1
        if session_id > 3:
          subject_id += 1
          session_id = 1

gt = list(itertools.chain.from_iterable(gt))
gt = [i.item() for i in gt]
results =  list(itertools.chain.from_iterable(results))
print(len(gt))
print(len(results))
mav = evaluate(results,gt)  
print('average precision: ',mav)

Loading  drive/My Drive/MERL Dataset/Data/32_1.npy
2106


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  from ipykernel import kernelapp as app


Len of original labels:  2106
Len of  predictions:  2040
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from abels
Removed from

In [0]:
len(gt)

3500

## Output 

In [0]:
def inferOnVideo(path_to_video):

  torch.cuda.empty_cache()
  path_to_weights = 'drive/My Drive/MERL Dataset/'
  model1 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
  model2 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
  model3 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
  model4 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
  model1, model2 = prepare_models(model1, model2, model3, model4,path_to_weights)
  torch.cuda.empty_cache()
  msn = MSN(spatial =model1, temporal=model2, spatial2=model3, temporal2=model4,        
            features_extraction_output=183296)
  msbl_model = bi_lstm(msn,batch_size = 20,num_layers = 1)
  torch.cuda.empty_cache()
  weights = torch.load(path_to_weights + '/Full Model2.pi')
  torch.cuda.empty_cache()
  msbl_model.load_state_dict(weights)
  del weights 
  torch.cuda.empty_cache()
  msbl_model.to(device)
  classification_model = msbl_model
  video, length = load_video_MS(path_to_video, half = False)
  test_dataloader = get_data_loader(video, batch_size = 20, with_labels = False)
  predictions = []
  probability = nn.Softmax()
  for x1,x2,x3,x4 in test_dataloader:
      with torch.no_grad():
        x = {1:x1.cuda() , 2:x2.cuda() , 3:x3.cuda() , 4:x4.cuda()}
        y_pred = classification_model.forward(x)
        del x
        torch.cuda.empty_cache()
        y_pred = y_pred.view(-1,6)
        y_pred = probability(y_pred)
        y_pred = y_pred.max(1)[1]
        y_pred = y_pred.cpu()
        torch.cuda.empty_cache()
        y_pred = list(chain.from_iterable(repeat(n, 7) for n in y_pred))
        predictions.append((y_pred))
  predictions = np.array(predictions)
  predictions = list(predictions.reshape(predictions.shape[0]*predictions.shape[1]))

  return predictions





In [0]:
video_path = 'drive/My Drive/MERL Dataset/Videos_MERL_Shopping_Dataset/10_1_crop.mp4'
result = inferOnVideo(video_path)

FileNotFoundError: ignored

In [0]:
  # path_to_video = 'drive/My Drive/MERL Dataset/Videos_MERL_Shopping_Dataset/10_1_crop.mp4'
  # torch.cuda.empty_cache()
  # path_to_weights = 'drive/My Drive/MERL Dataset/'
  # model1 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
  # model2 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
  # model3 = backbone_feature_extractor( features_extraction_output = 32768,classification = False)
  # model4 = backbone_feature_extractor( features_extraction_output = 150528,classification = False)
  # model1, model2 = prepare_models(model1, model2, model3, model4,path_to_weights)
  # torch.cuda.empty_cache()
  # msn = MSN(spatial =model1, temporal=model2, spatial2=model3, temporal2=model4,        
  #           features_extraction_output=183296)
  # msbl_model = bi_lstm(msn,batch_size = 20,num_layers = 1)
  # torch.cuda.empty_cache()
  # weights = torch.load(path_to_weights + '/Full Model2.pi')
  # torch.cuda.empty_cache()
  # msbl_model.load_state_dict(weights)
  # del weights 
  # torch.cuda.empty_cache()
  # msbl_model.to(device)
  # classification_model = msbl_model
  # video, length = load_video_MS(path_to_video, half = False)
  # test_dataloader = get_data_loader(video,labels = None, batch_size = 20, with_labels = False)
  predictions = []
  probability = nn.Softmax()
  for x1,x2,x3,x4 in test_dataloader:
      with torch.no_grad():
        x = {1:x1.cuda() , 2:x2.cuda() , 3:x3.cuda() , 4:x4.cuda()}
        y_pred = classification_model.forward(x)
        del x
        torch.cuda.empty_cache()
        y_pred = y_pred.view(-1,6)
        y_pred = probability(y_pred)
        y_pred = y_pred.max(1)[1]
        y_pred = y_pred.cpu()
        torch.cuda.empty_cache()
        y_pred = list(chain.from_iterable(repeat(n, 7) for n in y_pred))
        predictions.append((y_pred))
  predictions = np.array(predictions)
  predictions = list(predictions.reshape(predictions.shape[0]*predictions.shape[1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # Remove the CWD from sys.path while we load stuff.


In [0]:
np.array(predictions).shape

(4340,)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
data.shape

(356, 4)

## Writing predictions on Video

In [0]:
# Extract predictions
    output_frames = []
    for frame in video:
        image_tensor = Variable(transform(frame)).to(device)
        

        # Get label prediction for frame
        with torch.no_grad():
            prediction = model(image_tensor)
            predicted_label = labels[prediction.argmax(1).item()]

        # Draw label on frame
        d = ImageDraw.Draw(frame)
        d.text(xy=(10, 10), text=predicted_label, fill=(255, 255, 255))

        output_frames += [frame]

    # Create video from frames
    writer = skvideo.io.FFmpegWriter("output.gif")
    for frame in tqdm.tqdm(output_frames, desc="Writing to video"):
        writer.writeFrame(np.array(frame))
    writer.close()

In [0]:
path = 'drive/My Drive/MERL Dataset/Videos_MERL_Shopping_Dataset/10_2_crop.mp4'
%time video,lenght = load_video(path,resize_scale=0.5)

Loading  drive/My Drive/MERL Dataset/Videos_MERL_Shopping_Dataset/10_2_crop.mp4
Frame width: 920.0   frame_height: 680.0
resizing_width: 460    resizing_heighth: 340
6
END
4271
CPU times: user 5min, sys: 25.8 s, total: 5min 25s
Wall time: 4min 40s


In [0]:
subject_id = 21
session_id = 1 
threshold = 40
data_path ='drive/My Drive/MERL Dataset/Videos_MERL_Shopping_Dataset/'
label_path = 'drive/My Drive/MERL Dataset/Labels_MERL_Shopping_Dataset/'
data_save_path = 'drive/My Drive/MERL Dataset/Data_background_subtracted_OT/'
labels_save_path = 'drive/My Drive/MERL Dataset/Labels_background_subtracted_OT numpy/'
while subject_id < threshold:

            # Loading the videos and labels one at a time from the stored Numpy files
            
            path =  data_path + "/"+ str(subject_id) + "_" + str(session_id)+"_crop.mp4"
            if  os.path.exists(path): # check whether this file exists in the directory
              # Function load_video depends on the model
              # Here, we load the Data from pre-proccessed numpy files
              data  , length= load_video(video_path = path, background_sub=  True,object_trajectory = True)
              labels_path =  label_path +'/'+str(subject_id) +"_" + str(session_id)+"_label.mat"
              original_labels = get_video_label(labels_path,length)
              video_out = data_save_path +"/"+ str(subject_id) + "_" + str(session_id)+'.npy'
              labels_out = labels_save_path +'/'+str(subject_id) +"_" + str(session_id)+"_label.npy"
              np.save(video_out , data)
              np.save(labels_out, original_labels)
            else:
              print('Path does not exist: ', path)
            # Next Video    
            session_id += 1
            if session_id > 3:
              subject_id += 1
              session_id = 1
        

NameError: ignored

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU

# XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
def printm():
  GPUs = GPU.getGPUs()
  gpu = GPUs[0]
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 25.7 GB  | Proc size: 1.6 GB
GPU RAM Free: 11430MB | Used: 11MB | Util   0% | Total 11441MB
